# Training a YOLO model
___
Assuming miniconda3 was installed during the annotation step, let's set up a new environment, yolov8. The environment.yml file in this dir can be used to install the necessary packages. 

In a terminal navigate to this current directory and run the following:

<code>conda env create -f environment.yml</code>

Then select the yolov8 kernel for this notebook under 'Python environments'

In [ ]:
from ultralytics import YOLO
import ultralytics
import os

os.environ['WANDB_DISABLED'] = 'true'

There are two config yamls in this dir, practice_dataset.yaml and config.yaml.

practice_dataset.yaml defines the data location. Pointing YOLO to your data can be frustrating because there is a settings.yaml file (e.g. in ~/.config/Ultralytics) that specifies a root 'datasets_dir' that may confuse the path in your dataset.yaml. If the dataset is not found when you try to train the model, the error will specify where you need to look for the settings.yaml file. 

config.yaml is where you set metadata for the training run and hyperparameters. The file has descriptions of each of the settings, but here are some important ones:
- **model**: choose which pretrained yolo model to fine-tune (yolov8n = nano, yolov8s = small, etc.). Smaller models will train faster, larger models take longer to train and run, but are suited for very complex detection tasks. 
- **data**: make sure it points to the right dataset.yaml (practice_dataset.yaml in our case)
- **epochs**: One epoch is one train/val pass on the whole dataset. Start with just a few epochs (10) to troubleshoot your dataset/hyperparameter configuration, then train on 100-200 epochs when you have your settings dialed in
- **batch**: with a single GPU, Autobatch (-1) is great so you can set different img sizes and it'll adjust to the GPU available. Autobatch isn't available when you're training in parallel on multiple GPUs (afaik). The larger the batch size, the better - the algorith is 'learning' on more information each step. Your batch size will be limited by your GPU VRAM availability. 
- **imgsz**: imgs can be resized for more efficient training (imgsz: 640 resizes each img to 640x640). Smaller images mean more images per batch, but if the details we want to detect are small (flies, etc.), shrinking the size can obscure necessary details.
- **device**: useful if working with multiple GPUs or on a cluster
- **project**: name your project to keep it organized, especially if exporting results to WandB
- **optimizer**: you may want to specify SGD or one of the Adams based on the application, or experiment between several
- **close mosaic**: we'll get into the mosaic augmentation later, worth noting this setting
- **iou**: set the intersection over union threshold for Non-max Suppression (NMS). Decrease this if youre getting a lot of boxes duplicated over the same object. 

### Hyperparameters and Augmentations
Augmentations can be randomly applied to images during training to simulate image variation in real conditions (light source, different lenses, time of day, etc.). Augmenting images can make the model more robust to new data, but only use augmentations that make sense, e.g. don't vary the hue wildly if object color is important in classfying the object (fine if just detecting).
- **lr0 and lrf**: initial and final learning rates. important to experiment with if not automated
- **hsv_h, hsv_s, hsv_v**: vary the hue, saturation, and value (brightness) randomly by the percentage provided
- **scale, flipud, fliplr**: scale can be useful to account for changes in object size, flipping works well in the case of the fly or camera trap imgs because they are top-down (it wouldn't make sense to flipud an img of a giraffe).
- **mosaic**: a useful augmentation that makes a composite from the corners of 4 random images in the batch. This works well to shuffle the data even more during training and accommodate variation in unseen data.

Familiarize yourself with all the settings and consider their applicability to the dataset you're working on. 

In [ ]:
cfg = ultralytics.cfg.get_cfg(cfg='config.yaml') # Load in cfg
model = YOLO("yolov8n.pt") # Load pretrained model. It will download if it's not directly available

The following code trains the model. You'll likely have to adjust the root dataset path in the .config/Ultralytics/settings.yaml file to 

In [ ]:
results = model.train(cfg='config.yaml') # run training

The training metrics look pretty good, but go into the output dir (practice_runs/train#). The output dir contains the weights and metrics reports. Look through all the metric reports and get a sense for where the model is struggling.

The train_batch#.jpg and val_batch#_pred.jpg imgs are useful to see what the model is being trained on and get a small snapshot of how the model is doing on the val set. By comparing the val_labels and val_pred jpgs, we can see that very few insects in the val set are being detected. The Recall-confidence curve (R_curve) is also very poor, and the confusion_matrix shows that few insects are being detected. 

What does it mean when the training metrics are high but the validation performance is low?
___

We can apply our trained model to new images now. If your model isn't already defined (as it is during training above), you can load any model from the output weights:

In [ ]:
model = YOLO('practice_runs/train/weights/best.pt')

Then we can use the model to run inference on a new image. This image happens to work:

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# load in img
image_path = "../1_ANNOTATION/jpgs_to_annot/pi1_20240721_091114.jpg" 
image = cv2.imread(image_path)

# run model on image
results = model(image)

# plot results
for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1]) 
        plt.imshow(im)
plt.axis("off")
plt.show()